In [123]:
import tensorflow as tf
import numpy as np
import os
import time

In [124]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 
                                       'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


In [159]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [126]:
text[0:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [127]:
print(text[:1])


F


In [128]:
vocab = sorted(set(text))

In [129]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [130]:
text_as_int

array([18, 47, 56, ..., 45,  8,  0])

In [131]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [132]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))


'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [133]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [134]:
char_dataset

<TensorSliceDataset shapes: (), types: tf.int64>

In [135]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [136]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [137]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [138]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [139]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [140]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

# The embedding dimension
embedding_dim = 16

# Number of RNN units
rnn_units = 32

In [141]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [142]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [143]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [144]:
input_example_batch

<tf.Tensor: shape=(64, 100), dtype=int64, numpy=
array([[43,  1, 57, ...,  1, 50, 39],
       [56, 43, 39, ...,  0, 35, 46],
       [14, 59, 58, ..., 58, 53,  1],
       ...,
       [ 0, 16, 33, ...,  8,  0, 21],
       [10,  1, 58, ..., 57,  0, 46],
       [53, 51, 43, ..., 52, 39, 56]])>

In [145]:
model.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (64, None, 16)            1040      
_________________________________________________________________
gru_6 (GRU)                  (64, None, 32)            4800      
_________________________________________________________________
dense_6 (Dense)              (64, None, 65)            2145      
Total params: 7,985
Trainable params: 7,985
Non-trainable params: 0
_________________________________________________________________


In [146]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [147]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1745853


In [148]:
model.compile(optimizer='adam', loss=loss)


In [149]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [150]:
EPOCHS=10
EPOCHS=5
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Train for 172 steps
Epoch 1/5
172/172 [==============================] - 9s 53ms/step - loss: 3.4148
Epoch 2/5
172/172 [==============================] - 8s 45ms/step - loss: 2.7821
Epoch 3/5
172/172 [==============================] - 9s 53ms/step - loss: 2.5071
Epoch 4/5
172/172 [==============================] - 8s 46ms/step - loss: 2.3790
Epoch 5/5
172/172 [==============================] - 8s 47ms/step - loss: 2.2955


In [151]:
tf.train.latest_checkpoint(checkpoint_dir)


'./training_checkpoints/ckpt_5'

In [152]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (1, None, 16)             1040      
_________________________________________________________________
gru_7 (GRU)                  (1, None, 32)             4800      
_________________________________________________________________
dense_7 (Dense)              (1, None, 65)             2145      
Total params: 7,985
Trainable params: 7,985
Non-trainable params: 0
_________________________________________________________________


In [153]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [154]:
print(generate_text(model, start_string=u"ROMEO: "))

#print(generate_text(model, start_string=u"बुराइयाँ"))


ROMEO: il thes.
I dilch:
O swe, gunngilorans,
Whe ann hith You merey, se et tawc a we the lare alls othat be thitk faft whe
I bepry:
Rith thig ersinginmu heom? Ye tho thy hicl!

BIICoRHANI:
Ye bestt e ananls'ns'e hadile shange
dip for
Nhar thit! dey op:
Aliill wiad poiverer ouse yor didkt strintker by sre Int tem slegh he ut incot we?

DUTECLDD: I malg eve diver
Hesrt theer got he'te-d:
At, some hyer and yor fied mong.
Wanl sto moOMo haking kect srat arall well tamtros; in siplid nis int foar flomk'k schast Shere if nte, furd mele?

CRIREHASOR:
So lher ing wead mey our, kapbnik ard a loves yor, iik matth:
Whe foth, en
The loornouth s as cheale hy drattan, thy vet weid my hey shar sols, sherwe,
she prethespy'd as! I
Dor to u himee Fuite dowrit noch to noum whoud Kakeow,
Qeve thind
Th you wosre thomf batmann!
E Heurcowethes'ds goveror you put haviandt Won shis thad shaman howins: not uporlrorou:
Cecear a fun, gtate thave thunf.
An? venc;
I boke mear?

Sank your cy, po malt mepsary apeine

In [155]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [156]:
optimizer = tf.keras.optimizers.Adam()


In [157]:
@tf.function

def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [158]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.174643516540527
Epoch 1 Batch 100 Loss 3.2836732864379883
Epoch 1 Loss 3.1392
Time taken for 1 epoch 8.352621078491211 sec

Epoch 2 Batch 0 Loss 3.1401517391204834
Epoch 2 Batch 100 Loss 2.850127935409546
Epoch 2 Loss 2.7207
Time taken for 1 epoch 7.063384056091309 sec

Epoch 3 Batch 0 Loss 2.6965084075927734
Epoch 3 Batch 100 Loss 2.54624080657959
Epoch 3 Loss 2.4968
Time taken for 1 epoch 7.24484920501709 sec

Epoch 4 Batch 0 Loss 2.474186897277832
Epoch 4 Batch 100 Loss 2.4051477909088135
Epoch 4 Loss 2.3773
Time taken for 1 epoch 7.055747032165527 sec

Epoch 5 Batch 0 Loss 2.3469836711883545
Epoch 5 Batch 100 Loss 2.3307464122772217
Epoch 5 Loss 2.2583
Time taken for 1 epoch 7.063298225402832 sec

Epoch 6 Batch 0 Loss 2.280813217163086
Epoch 6 Batch 100 Loss 2.235830783843994
Epoch 6 Loss 2.2216
Time taken for 1 epoch 6.970677375793457 sec

Epoch 7 Batch 0 Loss 2.1874454021453857
Epoch 7 Batch 100 Loss 2.1787209510803223
Epoch 7 Loss 2.1737
Time taken for 1 e